In [52]:
import gurobipy as gp
from gurobipy import GRB
import pandas as pd 
import numpy as np


class OPTIMISER:
    
    def __init__(self):
        pass
    
    def example(self, a, objective, OC, OH):
        try:
            m = gp.Model("mip1")

            # Import data 
            path = "C:\\Users\\Arbeitsaccount\\Downloads\\SeminarCase1\\data_compos.xlsx"
            df2 = pd.read_excel(path)
            path = "C:\\Users\\Arbeitsaccount\\Downloads\\SeminarCase1\\Railway services-2024.xlsx"
            df3 = pd.read_excel(path)
            #df3 = df3.iloc[:-3]
            f = df2["Costs"]
            set = ["OH", "OC"]
            # Define the dimensions of the 3D matrix
            dim1 = 250
            dim2 = 200
            dim3 = 10

            # Create an empty 3D matrix (filled with uninitialized values)
            l = np.empty((dim1, dim2, dim3))
            s = np.empty((dim1, dim2, dim3))
            demand = np.empty((dim1, dim2))


            # Add values to specific elements of the matrix
            # matrix[0, 1, 2] = 10
            # matrix[1, 2, 3] = 20

            for t in range(250):
                for c in range(200):
                    # np.random.seed(30)
                    demand[t,c] = np.random.normal(df3["Demand(μ)"][c], df3["Demand(σ)"][c])
                    for p in range(10):
                        if demand[t,c] < df2["Capacity"][p]:
                            l[t,c,p] = 1
                            s[t,c,p] = 1
                        elif demand[t,c] == df2["Capacity"][p]:
                            l[t,c,p] = 0
                            s[t,c,p] = 1
                        else:
                            l[t,c,p] = 0
                            s[t,c,p] = 0
 

            x = {}
            for c in range(200):  # Define the first index range
                for p in range(10) :  # Define the second index range
                    x[c, p] = m.addVar(vtype=GRB.BINARY, name=f"x[{c},{p}]") 
            
            m.setObjective(gp.quicksum(x[c, p] * f[p] for c in range(200) for p in range(10)), GRB.MINIMIZE)
            
            for c in range(200):
                m.addConstr(gp.quicksum(x[c, p] for p in range(10)) == 1)
                if df3["Line"][c] == 400:
                    m.addConstr(gp.quicksum(x[c, p] * df2["Length"][p] for p in range(10)) <= 200)

            
            for u in set:
                for k in set:
                    if u != k:
                       m.addConstr(gp.quicksum(x[c,p] * df2[u][p] - x[c,p] * df2[k][p]  for c in range(200) for p in range(10)) <= gp.quicksum(x[c,p] * df2[k][p]  for c in range(200) for p in range(10)) * 0.25)

            

            m.addConstr(gp.quicksum(x[c,p] * l[t,c,p] for c in range(200) for t in range(250) for p in range(10)) >= 0.81 * 50000)

            for c in range(200):
                m.addConstr(gp.quicksum(x[c,p] * s[t,c,p] for t in range(250) for p in range(10)) >= 125)



            m.optimize()
            
            #for v in m.getVars():
            #    print("%s %g" % (v.varName, v.x))

            x_new = np.empty((dim2, dim3))
            for i in range(dim2):
                for j in range(dim3):
                    x_new[i, j] = x[i, j].x
            
            sumOC = 0
            sumOH = 0
            for c in range(200):
                tempSum = x_new[c] * df2["OC"]
                sumOC += tempSum
                tempSum = x_new[c] * df2["OH"]
                sumOH += tempSum
            OC[a] = np.sum(sumOC)
            OH[a] = np.sum(sumOH)

            objective[a] = m.objVal
            #print("Obj: %g" % m.objVal)
            
        except gp.GurobiError as e:
            print("Error code " + str(e.errno) + ": " + str(e))
        except AttributeError:
            print("Encountered an attribute error")
    
    def iterations(self, a):        
        objective = np.empty(a)
        OC = np.empty(a)
        OH = np.empty(a)
        for i in range(a):
            optimizer.example(i, objective, OC, OH)
        print("Obj mean: " + np.mean(objective).astype('str'))
        print("Obj stdev: " + np.std(objective).astype('str'))
        print("OC mean: " + np.mean(OC).astype('str'))
        print("OC stdev: " + np.std(OC).astype('str'))
        print("OH mean: " + np.mean(OH).astype('str'))
        print("OH stdev: " + np.std(OH).astype('str'))
        print(objective)

if __name__ == "__main__":
    optimizer = OPTIMISER()
    numOfIterations = 2
    optimizer.iterations(numOfIterations)


Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (win64 - Windows 10.0 (19045.2))

CPU model: Intel(R) Core(TM) i5-7200U CPU @ 2.50GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads

Optimize a model with 427 rows, 2000 columns and 10060 nonzeros
Model fingerprint: 0xa414e938
Variable types: 0 continuous, 2000 integer (2000 binary)
Coefficient statistics:
  Matrix range     [3e-01, 3e+02]
  Objective range  [2e+05, 8e+05]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 4e+04]
Presolve removed 227 rows and 536 columns
Presolve time: 0.03s
Presolved: 200 rows, 1464 columns, 5275 nonzeros
Variable types: 0 continuous, 1464 integer (1464 binary)
Found heuristic solution: objective 7.989000e+07

Root relaxation: objective 7.531808e+07, 249 iterations, 0.03 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap |